# Preparación de datos: etiquetas de actividades por video

Este notebook procesa el JSON exportado de Label Studio y genera un archivo tabular (CSV) con las etiquetas de actividad por rango de frames para cada video.

In [6]:
import json
import pandas as pd
from pathlib import Path

# Ruta al archivo JSON exportado de Label Studio
json_path = Path('../results/project-1-at-2025-10-17-18-57-0eeb7784.json')

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"Total de videos anotados: {len(data)}")

Total de videos anotados: 11


In [7]:
# Extraer etiquetas por video y rango de frames
data_rows = []

for item in data:
    video_file = item.get('file_upload', '')
    annotations = item.get('annotations', [])
    if not annotations:
        continue
    # Tomar la primera anotación (asumimos una por video)
    results = annotations[0].get('result', [])
    for r in results:
        label = r['value']['timelinelabels'][0]
        for rng in r['value']['ranges']:
            start = rng['start']
            end = rng['end']
            data_rows.append({
                'video': video_file,
                'start_frame': start,
                'end_frame': end,
                'actividad': label
            })

df = pd.DataFrame(data_rows)
df.head()

,video,start_frame,end_frame,actividad
0,a327f941-Video_1.mp4,18,37,parandose
1,a327f941-Video_1.mp4,1,18,sentado
2,a327f941-Video_1.mp4,37,40,parado
3,a327f941-Video_1.mp4,40,119,caminando adelante
4,a327f941-Video_1.mp4,120,155,girando


In [8]:
# Guardar el DataFrame como CSV para el pipeline
output_path = Path('../results/labels_por_rango.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False, encoding='utf-8')
print(f"CSV guardado en: {output_path}")

# Resumen rápido
print("\nActividades encontradas:")
print(df['actividad'].value_counts())
print("\nVideos únicos:")
print(df['video'].nunique())

CSV guardado en: ..\results\labels_por_rango.csv

Actividades encontradas:
actividad
parado                28
girando               23
sentado               22
parandose             19
caminando adelante    13
sentandose            11
caminado atras        10
Name: count, dtype: int64

Videos únicos:
11
